In [1]:
import os
import json
import cv2

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from utils import label_accuracy_score
import cv2

import numpy as np
import pandas as pd

# 전처리를 위한 라이브러리
from pycocotools.coco import COCO
import torchvision
import torchvision.transforms as transforms

import albumentations as A
from albumentations import *
from albumentations.pytorch import ToTensorV2

# 시각화를 위한 라이브러리
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

plt.rcParams['axes.grid'] = False

In [ ]:
root_dir = "/opt/ml/submission/"

data1_file = "6065.csv"
data2_file = "6063.csv"
data3_file = "5997.csv"
data4_file = "5996.csv"

data1 = pd.read_csv(root_dir + data1_file)
data2 = pd.read_csv(root_dir + data2_file)
data3 = pd.read_csv(root_dir + data3_file)
data4 = pd.read_csv(root_dir + data4_file)

In [2]:
len(data1)

In [45]:
str1 = data1.iloc[0]['PredictionString'].split(' ')

In [3]:
len(str1)

In [4]:
print(256 * 256)

## 데이터 loader작동 확인

In [48]:
%matplotlib inline

dataset_path = '/opt/ml/input/data'
anns_file_path = dataset_path + '/' + 'train.json'


with open(anns_file_path, 'r') as f:
    dataset = json.loads(f.read())

categories = dataset['categories']
anns = dataset['annotations']
imgs = dataset['images']
nr_cats = len(categories)
nr_annotations = len(anns)
nr_images = len(imgs)


cat_names = []
super_cat_names = []
super_cat_ids = {}
super_cat_last_name = ''
nr_super_cats = 0
for cat_it in categories:
    cat_names.append(cat_it['name'])
    super_cat_name = cat_it['supercategory']

    if super_cat_name != super_cat_last_name:
        super_cat_names.append(super_cat_name)
        super_cat_ids[super_cat_name] = nr_super_cats
        super_cat_last_name = super_cat_name
        nr_super_cats += 1

In [49]:
cat_histogram = np.zeros(nr_cats,dtype=int)
for ann in anns:
    cat_histogram[ann['category_id']] += 1

df = pd.DataFrame({'Categories': cat_names, 'Number of annotations': cat_histogram})
df = df.sort_values('Number of annotations', 0, False)

In [50]:
sorted_temp_df = df.sort_index()

# background = 0 에 해당되는 label 추가 후 기존들을 모두 label + 1 로 설정
sorted_df = pd.DataFrame(["Backgroud"], columns = ["Categories"])
sorted_df = sorted_df.append(sorted_temp_df, ignore_index=True)

In [5]:
# class (Categories) 에 따른 index 확인 (0~11 : 총 12개)
sorted_df

In [52]:
dataset_path = '/opt/ml/input/data'

def data_eda():
    """데이터 eda 및 학습 데이터에 필요한 데이터 프레임 가져오기 

    Returns:
        sorted_df(DataFrame) : 로드 시 필요정보를 가지는 데이터프레임 반환
    """

    anns_file_path = dataset_path + '/' + 'train.json'
    
    # 어노테이션 정보 가져오기
    with open(anns_file_path, 'r') as f:
        dataset = json.loads(f.read())
        
    categories = dataset['categories']
    anns = dataset['annotations']
    imgs = dataset['images']
    nr_cats = len(categories)
    nr_annotations = len(anns)
    nr_images = len(imgs)
    
    cat_names = []
    super_cat_names = []
    super_cat_ids = {}
    super_cat_last_name = ''
    nr_super_cats = 0
    
    for cat_it in categories:
        cat_names.append(cat_it['name'])
        super_cat_name = cat_it['supercategory']
        
        if super_cat_name != super_cat_last_name:
            super_cat_names.append(super_cat_name)
            super_cat_ids[super_cat_name] = nr_super_cats
            super_cat_last_name = super_cat_name
            nr_super_cats += 1
            
    cat_histogram = np.zeros(nr_cats,dtype=int)
    for ann in anns:
        cat_histogram[ann['category_id']] += 1
        
    
    df = pd.DataFrame({'Categories': cat_names, 'Number of annotations': cat_histogram})
    df = df.sort_values('Number of annotations', 0, False)

    sorted_temp_df = df.sort_index()
    
    # background = 0 에 해당되는 label 추가 후 기존들을 모두 label + 1 로 설정
    sorted_df = pd.DataFrame(["Backgroud"], columns = ["Categories"])
    sorted_df = sorted_df.append(sorted_temp_df, ignore_index=True)

    return sorted_df

def get_classname(classID, cats):
    for i in range(len(cats)):
        if cats[i]['id']==classID:
            return cats[i]['name']
    return "None"

class CustomDataLoader(Dataset):
    """ dataloader의 정의
    """
    def __init__(self, data_dir, mode = 'train', transform = None, predict = None):
        super().__init__()
        self.mode = mode
        self.transform = transform
        self.coco = COCO(data_dir)
        sorted_df = data_eda()
        self.category_names = list(sorted_df.Categories)
        self.predict = predict
        
    def __getitem__(self, index: int):
        # dataset이 index되어 list처럼 동작
        image_id = self.coco.getImgIds(imgIds=index)
        image_infos = self.coco.loadImgs(image_id)[0]
        
        # cv2 를 활용하여 image 불러오기
        paths = os.path.join(dataset_path, image_infos['file_name'])
        images = cv2.imread(os.path.join(dataset_path, image_infos['file_name']))
        images = cv2.cvtColor(images, cv2.COLOR_BGR2RGB).astype(np.float32)
        images /= 255.0
        
        if (self.mode in ('train', 'val')):
            ann_ids = self.coco.getAnnIds(imgIds=image_infos['id'])
            anns = self.coco.loadAnns(ann_ids)

            cat_ids = self.coco.getCatIds()
            cats = self.coco.loadCats(cat_ids)

            # masks : size가 (height x width)인 2D
            # 각각의 pixel 값에는 "category id + 1" 할당
            # Background = 0
            masks = np.zeros((image_infos["height"], image_infos["width"]))
            # Unknown = 1, General trash = 2, ... , Cigarette = 11
            for i in range(len(anns)):
                className = get_classname(anns[i]['category_id'], cats)
                pixel_value = self.category_names.index(className)
                masks = np.maximum(self.coco.annToMask(anns[i])*pixel_value, masks)
            masks = masks.astype(np.float32)

            # transform -> albumentations 라이브러리 활용
            if self.transform is not None:
                transformed = self.transform(image=images, mask=masks)
                images = transformed["image"]
                masks = transformed["mask"]
            
            return paths, images, masks, image_infos
        
        if self.mode == 'test':
            # transform -> albumentations 라이브러리 활용
            if self.transform is not None:
                transformed = self.transform(image=images)
                images = transformed["image"]
            
            return paths,images, image_infos

        if self.mode == "pesudo_label":
            images = cv2.imread(os.path.join(dataset_path, self.predict.iloc[index]['image_id']))
            images = cv2.cvtColor(images, cv2.COLOR_BGR2RGB).astype(np.float32)
            images /= 255.0


            if self.transform is not None:
                transformed = self.transform(image=images)
                images = transformed["image"]

            
            masks = self.predict.iloc[index]['PredictionString'].split(' ')
            masks = np.array(masks, dtype = float)
            masks = masks.reshape(256,256)
            masks = torch.FloatTensor(masks)
            
            return images, masks
    
    
    def __len__(self) -> int:
        # 전체 dataset의 size를 return
        return len(self.coco.getImgIds())

In [6]:
dataset_path = '/opt/ml/input/data'
test_path = dataset_path + '/test.json'

# 배치를 위한 필요 함수
def collate_fn(batch):
    return tuple(zip(*batch))

test_transform = A.Compose([A.Resize(256, 256),ToTensorV2()])


# test 데이터셋
test_dataset = CustomDataLoader(data_dir=test_path, mode='pesudo_label', transform=test_transform, predict = data1)



test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=3,
                                          num_workers=4,
                                          collate_fn=collate_fn)

In [54]:
for images, masks in test_loader:
    temp_images = images
    temp_masks = masks
    
    break

In [7]:
len(temp_images)

In [8]:
len(temp_masks)

In [58]:
batch_size = 16   # Mini-batch size
num_epochs = 20
learning_rate = 0.0001

In [9]:
test_path = dataset_path + '/test.json'

# collate_fn needs for batch
def collate_fn(batch):
    return tuple(zip(*batch))

test_transform = A.Compose([
    A.Resize(256, 256),
    ToTensorV2()
])

# test dataset
test_dataset = CustomDataLoader(data_dir=test_path, mode='test', transform=test_transform)



test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=64,
                                          num_workers=1,
                                          collate_fn=collate_fn)

In [60]:
mask1_list = []
mask2_list = []
mask3_list = []
mask4_list = []

for i in range(64):
    temp = data1.iloc[i]['PredictionString'].split(' ')
    
    temp = np.array(temp, dtype = float)
    temp = temp.reshape(256,256)
    temp = torch.FloatTensor(temp)
    
    mask1_list.append(temp)
    
for i in range(64):
    temp = data2.iloc[i]['PredictionString'].split(' ')
    
    temp = np.array(temp, dtype = float)
    temp = temp.reshape(256,256)
    temp = torch.FloatTensor(temp)
    
    mask2_list.append(temp)
    
for i in range(64):
    temp = data3.iloc[i]['PredictionString'].split(' ')
    
    temp = np.array(temp, dtype = float)
    temp = temp.reshape(256,256)
    temp = torch.FloatTensor(temp)
    
    mask3_list.append(temp)
    
for i in range(64):
    temp = data4.iloc[i]['PredictionString'].split(' ')
    
    temp = np.array(temp, dtype = float)
    temp = temp.reshape(256,256)
    temp = torch.FloatTensor(temp)
    
    mask4_list.append(temp)
    

In [61]:
# 첫번째 batch의 추론 결과 확인
for paths,imgs, image_infos in test_loader:
    temp_paths = paths
    image_infos = image_infos
    temp_images = imgs
    
    break

## 결과  시각화 분석 

In [10]:
for i in range(64):
    fig, (ax1, ax2, ax3, ax4, ax5) = plt.subplots(nrows=1, ncols=5, figsize=(25, 6), sharey=True)

    # Original image
    ax1.imshow(temp_images[i].permute([1,2,0]),vmin = 0, vmax=11)
    ax1.grid(False)
    ax1.set_title(temp_paths[i], fontsize = 10)

    ax2.imshow(mask1_list[i],vmin = 0, vmax=11)
    ax2.grid(False)
    ax2.set_title(data1_file, fontsize = 10)

    ax3.imshow(mask2_list[i],vmin = 0,vmax=11)
    ax3.grid(False)
    ax3.set_title(data2_file, fontsize = 10)
    
    ax4.imshow(mask3_list[i],vmin = 0,vmax=11)
    ax4.grid(False)
    ax4.set_title(data3_file, fontsize = 10)
    
    axes_img = ax5.imshow(mask4_list[i],vmin = 0,vmax=11)
    ax5.grid(False)
    ax5.set_title(data4_file, fontsize = 10)
    
    fig.subplots_adjust(right=0.8)
    cbar_ax = fig.add_axes([0.85, 0.15, 0.03, 0.7])
    fig.colorbar( axes_img, cax=cbar_ax)
    plt.show()